# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import pandas as pd
import numpy as np

Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('https://github.com/collab-uniba/PanelJamDataExtractor/blob/master/data/TabellaCompleta.xlsx?raw=true', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**panel_author** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [18]:
#time = table['time'].tolist()
#table['time'] = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','panel_author','time','author_stars'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,panel_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,tot_views_author,author_ranking,panel_stars
panel_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.05,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.11,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,0.01,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.63,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.76,0.31,0.19
tot_views_author,0.05,-0.03,0.11,0.01,0.63,0.76,1.00,0.63,0.12
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,0.63,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,0.12,1.00


Come si può osservare dalla matrice, c'è un alto grado di correlazione tra **tot_loves_author** e **author_stars** (**0.76**) superiore al limite di **0.7**. Questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking d'autore, per questo si decide di eliminarle entrambe. 

In [20]:
table = table.drop(columns = ['tot_views_author','tot_loves_author'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,panel_depth,extended,has_avatar_author,has_bio_author,author_followers,author_ranking,panel_stars
panel_depth,1.00,-0.42,0.06,0.03,0.02,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.33,0.18
author_ranking,0.07,-0.04,0.09,-0.03,0.33,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.12,1.00


# MODELLO DI REGRESSIONE

Dopo aver costruito la matrice di correlazione ed eliminato eventuali problemi di collinearità, si procede con la costruzione del modello di regressione. Il primo passaggio consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [22]:
featureTable = table
logit = featureTable.drop(columns = ['panel_depth','author_followers',
                'author_ranking','panel_stars'])
numerical = featureTable.drop(columns = ['extended','has_avatar_author','has_bio_author'])

bisogna effettuare una log-normalizzazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica.

In [23]:
normTable = np.log1p(numerical)
normTable['extended'] = logit['extended']
normTable['has_avatar_author'] = logit['has_avatar_author']
normTable['has_bio_author'] = logit['has_bio_author']

Per costruire il modello di regressione si è utilizzata la funzione **glm** del package **statsmodel**, alla quale sono stati passati in input 3 parametri.

-**formula**: prima del simbolo tilde (~) si deve specificare la variabile dipendente, mentre dopo bisogna specificare le variabili indipendenti, i quali costituiranno i predittori del modello.

-**data**: la tabella da cui estrarre i valori delle variabili indipendenti.

-**family**: specifica il modello di regressione che si vuole andare a costruire, in questo caso binomial ed è specificato da **sm.families.Binomial()**.

In [24]:
formula = """extended ~ panel_depth + author_followers +
             has_bio_author + has_avatar_author + author_ranking + panel_stars"""

model = smf.glm(formula = formula, data = normTable,
                family=sm.families.Binomial(statsmodels.genmod.families.links.logit)).fit()
print(model.summary())

                           Generalized Linear Model Regression Results                           
Dep. Variable:     ['extended[False]', 'extended[True]']   No. Observations:                13748
Model:                                               GLM   Df Residuals:                    13741
Model Family:                                   Binomial   Df Model:                            6
Link Function:                                     logit   Scale:                          1.0000
Method:                                             IRLS   Log-Likelihood:                -5334.3
Date:                                   Wed, 17 Jun 2020   Deviance:                       10669.
Time:                                           19:37:55   Pearson chi2:                 1.59e+04
No. Iterations:                                        6                                         
Covariance Type:                               nonrobust                                         
                    

d:\dev-cpp\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  after removing the cwd from sys.path.


# CALCOLO ODDS RATIO E CONCLUSIONI

## Che cos'è l'odds ratio

l'**Odds Ratio**(**OR**) è un modo per rappresentare la forza dell'associazione tra fattori di esposizione (nel nostro caso i predittori) e l'esito o risultato(ovvero la variabile dipendente). Possiamo distinguere 3 casi di OR:

- Un OR > 1 significa maggiore probabilità di associazione tra l'esposizione e il risultato. Nello specifico, un OR di 1.2 significa che c'è un aumento del 20% delle probabilità di un risultato con una data esposizione. In questo caso significa che se un predittore dovesse avere un OR di 1.2, si avrebbe un aumento del 20% della probabilità che un panel venga esteso(dato che la variabile dipendente è extended) con quel predittore.

- Un OR < 1 significa minore probabilità di associazione tra l'esposizione e il risultato. Nello specifico, un OR di 0.2 significa che c'è una diminuzione dell' 80% delle probabilità di un risultato con una data esposizione. In questo caso significa che se un predittore dovesse avere un OR di 0.2, si avrebbe una diminuzione dell' 80% della probabilità che un panel venga esteso con quel predittore.

- Un OR = 1 significa che non c'è nessuna associazione tra l'esposizione e il risultato.

**ATTENZIONE!**
Un OR pari a 0.2 non implica che c'è un aumento del 20% di probabilità che un panel venga esteso con un determinato predittore, stesso discorso vale per un OR pari a 1.2, con ciò non significa che c'è una diminuzione dell' 80% che un panel non venga esteso. Un predittore con OR < 1 influisce **solo** negativamente sul risultato, mentre un  OR > 1 influisce **solo** positivamente. 

Per calcolare l'OR di ogni predittore, è stata utilizzata la funzione **exp** del modulo **numpy** la quale restituisce l'esponenziale del coefficiente di ogni predittore. Successivamente è stata creata una tabella che contenesse i predittori, i valori di OR e l'intervallo di confidenza.

In [26]:
oddsRatio = np.exp(model.params.values)
confInt = list(model.conf_int().values)
confInt = list(np.exp(confInt))

data = {'features': ['Intercept','has_bio_author_TRUE','has_avatar_author_TRUE','panel_depth','author_followers',
                     'author_ranking','panel_stars'],
        'Odds Ratio': oddsRatio,
        'Confidence Interval': confInt}
oddsRatioTable = pd.DataFrame(data,columns = ['features','Odds Ratio','Confidence Interval'])
oddsRatioTable

,features,Odds Ratio,Confidence Interval
0,Intercept,0.001362,"[0.000887499876057039, 0.0020911231434274273]"
1,has_bio_author_TRUE,1.101035,"[0.9857114345152791, 1.2298508522973128]"
2,has_avatar_author_TRUE,0.905009,"[0.6816446496709746, 1.2015660585150643]"
3,panel_depth,49.095253,"[41.67113232479451, 57.842053346645564]"
4,author_followers,0.984289,"[0.9412635990248716, 1.029282097168438]"
5,author_ranking,0.906319,"[0.8635518021298101, 0.9512042545912915]"
6,panel_stars,1.844507,"[1.7187962631854603, 1.9794120129143495]"


Prima di analizzare i risultati ottenuti bisogna specificare che cos'è un intervallo di confidenza e perchè è importante.
Quando si effettua la stima di un parametro, in questo caso il coefficiente di un predittore, è opportuno accompagnare tale stima da un range di valori plausibili per quel parametro, il quale è appunto l'**intervallo di confidenza**. Ad ogni intervallo di confidenza viene associato un livello di confidenza che rappresenta il grado di attendibilità dell'intervallo. In questo caso è stato impostato un grado di confidenza del 95%, esso indica la percentuale con la quale il valore reale del parametro è compreso tra l'estremo inferiore e l'estremo superiore dell'intervallo. Dopo aver calcolato tutti i valori di OR, si deve anche calcolare l'intervallo di confidenza al 95% di questi ultimi, questo perchè, se nell'intervallo di confidenza di un OR è compreso il valore 1, allora il predittore a cui fa riferimento l'OR, è considerato **statisticamente non significativo**. Alla luce di queste considerazioni si può affermare che:

- La variabile **has_bio_author_TRUE** è statisticamente non significativa in quanto contiene il valore 1 nel suo intervallo di confidenza **[0.986, 1.230]**.
- La variabile **has_avatar_author_TRUE** è statisticamente non significativa in quanto contiene il valore 1 nel suo intervallo **[0.682, 1.120]**.
- La variabile **panel_depth** è statisticamente significativa ed ha un valore di OR pari a **49.09**. Questo implica che esso è un predittore estremamente influente sulla variabile dipendente, infatti aumenta la probabilità di estensione di un panel del **~4800%**.
- La variabile **author_followers** è statisticamente non significativa in quanto contiene il valore 1 nel suo intervallo di confidenza **[0.941,1.029]**.
- La variabile **author_ranking** è statisticamente significativa ed ha un valore di OR pari a **0.90**. Questo implica che esso è un predittore che, seppure di poco, influisce negativamente sulla variabile dipendente. Infatti diminiuisce di circa il **10%** la probabilità di un panel di essere esteso.
- La variabile **panel_stars** è statisticamente significativa ed ha un valore di OR pari a **1.84**. Questo implica che esso è un predittore molto influente sulla variabile dipendente, infatti aumenta la probabilita di estensione di un panel del **84%**.